# 1.Import dependencies

In [4]:
import cv2
import joblib
import numpy as np

face_classifer = joblib.load("../../models/Face Detector/FaceClassifier_HoG(64, 64)(16, 16)(8, 8)(0.95).pkl")

# 3. Define Helper Functions

In [7]:
win_size = (64, 64) 
block_size = (16, 16)
block_stride = (8, 8)
cell_size = (8, 8)
num_bins = 9

HOG = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, num_bins)



def extract_features_face_detection(Image):

    gray_image = cv2.cvtColor(Image, cv2.COLOR_BGR2GRAY)

    equalized_image = cv2.equalizeHist(gray_image)

    resized_image = cv2.resize(equalized_image, win_size)
        
    feature = HOG.compute(resized_image).flatten()

    if feature.max() > 0:
        feature /= feature.max()

    return feature



def union_boxes_min_max(boxes, min_x, min_y, max_x, max_y):
  overall_box = None

  if not overall_box:
    for box in boxes:
      x_min, y_min, x_max, y_max = box
      min_x = min(min_x, x_min)
      min_y = min(min_y, y_min)
      max_x = max(max_x, x_max)
      max_y = max(max_y, y_max)
    overall_box = (min_x, min_y, max_x, max_y)
    
    return overall_box



def filter_boxes_with_skin_mask(boxes, skin_mask):
  filtered_boxes = []
  for box in boxes:
    x_min, y_min, x_max, y_max = box
    
    box_area = (x_max - x_min) * (y_max - y_min)
    
    intersection_mask = skin_mask[y_min:y_max, x_min:x_max]
    intersection_area = cv2.countNonZero(intersection_mask)
    
    iou = intersection_area / box_area

    # Define threshold for acceptable overlap with skin mask
    threshold = 0.5

    if iou >= threshold:
      filtered_boxes.append(box)

  return filtered_boxes  



def get_bounding_box(mask):
    non_zero_pixels = np.nonzero(mask)

    min_row, min_col = np.min(non_zero_pixels, axis=1)
    max_row, max_col = np.max(non_zero_pixels, axis=1)

    # Return coordinates in desired format (x0, y0, x1, y1)
    return min_col, min_row, max_col, max_row 



def segment_skin(image):
  ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

  lower_thresh = (0, 133, 77)
  upper_thresh = (255, 173, 127)

  skin_mask = cv2.inRange(ycrcb, lower_thresh, upper_thresh)

  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
  skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_OPEN, kernel, iterations=3)

  return skin_mask



def face_detection(frame) -> list:
    
    detected_faces = []

    window_sizes = [(64, 64), (96, 96)]

    skin_mask = segment_skin(frame)

    x0, y0, x1, y1 = get_bounding_box(skin_mask)

    for window_size in window_sizes:

        y_stride = window_size[0]
        x_stride = window_size[1]

        for y in range(y0, y1, y_stride):
            for x in range(x0, x1, x_stride):
                window = frame[y : y+window_size[0], x : x+window_size[1]]

                [prediction] = face_classifer.predict( [extract_features_face_detection(window)] )

                if prediction == 1:  
                    detected_faces.append([x, y, x + window_size[1], y + window_size[0]])

    detected_faces = filter_boxes_with_skin_mask(detected_faces, skin_mask)
    detected_faces = union_boxes_min_max(detected_faces, x0, y0, x1, y1)

    return detected_faces

# 4. Main Application

In [8]:
camera = cv2.VideoCapture(0)

while camera.isOpened():

    ret, frame = camera.read()

    face = face_detection(frame) # face -> (x0, y0, x1, y1)

    cv2.rectangle(frame, (face[0], face[1]), (face[2], face[3]), (0, 255, 0), 2)

    cv2.imshow("Stream", frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        print(frame.shape)
        break

camera.release()
cv2.destroyAllWindows()

(480, 640, 3)
